In [13]:
import numpy as np
import pandas as pd
from docplex.mp.model import Model
import os
# import dimod
# from dwave.system import LeapHybridSampler

In [15]:
# --- Generate couplings ---
def generate_J(N, seed=None):
    # Create directory if it doesn't exist
    os.makedirs('generated_data', exist_ok=True)
    
    if seed is not None:
        np.random.seed(seed)
    std = 1.0 / np.sqrt(N)
    data = [(i, j, np.random.normal(0, std)) for i in range(N) for j in range(i+1, N)]
    df = pd.DataFrame(data, columns=['i','j','Jij'])
    
    if seed is not None:
        filepath = f'generated_data/Jij_N{N}_seed{seed}.csv'
        df.to_csv(filepath, index=False)
        print(f"Saved couplings for N={N} to {filepath}")
    else:
        filepath = f'generated_data/Jij_N{N}.csv'
        df.to_csv(filepath, index=False)
        print(f"Saved couplings for N={N} to {filepath}")

# --- Generate perturbed couplings (Goal 3) ---
def generate_J_perturbed(N, seed_orig, seed_pert, epsilon=0.01):
    """
    Generate perturbed couplings: J_ij^pert = J_ij^orig + epsilon * delta_J_ij
    where delta_J_ij is independent noise with same distribution as J_ij.
    
    Args:
        N: System size
        seed_orig: Seed for original couplings (must already exist)
        seed_pert: Seed for independent perturbation noise
        epsilon: Perturbation strength (default 0.01)
    """
    os.makedirs('generated_data', exist_ok=True)
    
    # Load original couplings
    df_orig = pd.read_csv(f'generated_data/Jij_N{N}_seed{seed_orig}.csv')
    
    # Generate independent perturbation noise
    np.random.seed(seed_pert)
    std = 1.0 / np.sqrt(N)
    delta_J = [np.random.normal(0, std) for _ in range(len(df_orig))]
    
    # Create perturbed couplings
    df_pert = df_orig.copy()
    df_pert['Jij'] = df_orig['Jij'] + epsilon * np.array(delta_J)
    
    filepath = f'generated_data/Jij_N{N}_seed{seed_orig}_pert{seed_pert}_eps{epsilon}.csv'
    df_pert.to_csv(filepath, index=False)
    print(f"Saved perturbed couplings to {filepath}")
    return df_pert

# --- Load couplings ---
def load_J(N, seed=None, seed_pert=None, epsilon=0.01):
    if seed_pert is not None:
        # Load perturbed couplings
        filename = f'generated_data/Jij_N{N}_seed{seed}_pert{seed_pert}_eps{epsilon}.csv'
    elif seed is not None:
        # Load original couplings
        filename = f'generated_data/Jij_N{N}_seed{seed}.csv'
    else:
        filename = f'generated_data/Jij_N{N}.csv'
    
    df = pd.read_csv(filename)
    N = max(df['i'].max(), df['j'].max()) + 1
    return [(int(i), int(j), float(Jij)) for i,j,Jij in df.values], N

In [16]:
def solve_SK(rows, N, seed, time_limit=None, perturbed=False, seed_pert=None, epsilon=0.01):
    from docplex.mp.model import Model
    import pandas as pd
    import os
    
    # Create directory if it doesn't exist
    os.makedirs('cplex_solutions', exist_ok=True)

    mdl = Model('SK_Ising')
    x = mdl.binary_var_list(N, name='x')

    expr = 0
    for i, j, Jij in rows:
        expr += -4*Jij * x[i]*x[j] + 2*Jij*(x[i] + x[j]) - Jij

    mdl.minimize(expr)

    if time_limit is not None:
        mdl.parameters.timelimit = float(time_limit)

    sol = mdl.solve(log_output=False)

    if not sol:
        print("No solution found.")
        return

    x_sol = [int(sol.get_value(var)) for var in x]
    S = [2*xi - 1 for xi in x_sol]
    E = sum(-Jij * S[i]*S[j] for i,j,Jij in rows)

    # Print solution status and gap
    status = mdl.solve_details.status
    mip_gap = mdl.solve_details.mip_relative_gap

    print(f"N = {N}\nEnergy = {E}")
    print(f"Status: {status}")
    if mip_gap is not None:
        print(f"MIP gap: {mip_gap:.4f}")

    # Save with appropriate filename
    if perturbed and seed_pert is not None:
        filepath = f'cplex_solutions/solution_N{N}_seed{seed}_pert{seed_pert}_eps{epsilon}.csv'
    else:
        filepath = f'cplex_solutions/solution_N{N}_seed{seed}.csv'
    
    pd.DataFrame({'i': range(N), 'Energy_cplex': E, 'x': x_sol, 'S': S}).to_csv(filepath, index=False)
    print(f"Saved solution to {filepath}")
    return E, S

## Goal 3: Chaos Exponent - Original Ground States

First, compute ground states for original (unperturbed) couplings.

In [10]:
# Goal 3: Compute original ground states (already done above, but rerun if needed)
N = 50
M = 40  # Number of disorder realizations for Goal 3

# Only run if solutions don't exist
energy_orig_list = []
for i in range(M):
    # Check if solution already exists
    filepath = f'cplex_solutions/solution_N{N}_seed{i}.csv'
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        E_orig = df['Energy_cplex'][0]
        print(f"Loaded existing solution for seed {i}: E = {E_orig}")
    else:
        # Generate and solve if not exists
        generate_J(N, seed=i)
        rows, N = load_J(N, seed=i)
        E_orig, S_orig = solve_SK(rows, N, seed=i, time_limit=300)
    
    energy_orig_list.append(E_orig)

print(f"\nOriginal energies computed for {M} realizations.")
print(f"Mean E/N = {np.mean(energy_orig_list)/N:.4f}")

Saved couplings for N=50 to generated_data/Jij_N50_seed0.csv
N = 50
Energy = -33.952800215526594
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed0.csv
Saved couplings for N=50 to generated_data/Jij_N50_seed1.csv
N = 50
Energy = -33.952800215526594
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed0.csv
Saved couplings for N=50 to generated_data/Jij_N50_seed1.csv
N = 50
Energy = -36.2219649608972
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed1.csv
Saved couplings for N=50 to generated_data/Jij_N50_seed2.csv
N = 50
Energy = -36.2219649608972
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed1.csv
Saved couplings for N=50 to generated_data/Jij_N50_seed2.csv
N = 50
Energy = -34.954109670887355
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/so

## Goal 3: Perturbed Ground States

Generate perturbed couplings and compute ground states: $J_{ij}^{\text{pert}} = J_{ij}^{\text{orig}} + \epsilon \cdot \delta J_{ij}$

In [11]:
# Goal 3: Compute perturbed ground states
# Option A: M=20 different disorder realizations, each with one independent perturbation
epsilon = 0.01

energy_pert_list = []
delta_E_list = []

for i in range(M):
    # Use different perturbation seed for each disorder (seed_pert = 10000 + i)
    # This ensures independent perturbation noise for each base disorder
    seed_pert = 10000 + i
    
    # Generate perturbed couplings
    generate_J_perturbed(N, seed_orig=i, seed_pert=seed_pert, epsilon=epsilon)
    
    # Load and solve perturbed instance
    rows_pert, N = load_J(N, seed=i, seed_pert=seed_pert, epsilon=epsilon)
    E_pert, S_pert = solve_SK(rows_pert, N, seed=i, time_limit=300, 
                               perturbed=True, seed_pert=seed_pert, epsilon=epsilon)
    
    energy_pert_list.append(E_pert)
    
    # Compute energy change
    E_orig = energy_orig_list[i]
    delta_E = abs(E_pert - E_orig)
    delta_E_list.append(delta_E)
    
    print(f"Seed {i}, Pert {seed_pert}: E_orig = {E_orig:.4f}, E_pert = {E_pert:.4f}, |ΔE| = {delta_E:.4f}")

# Compute RMS energy change (disorder-averaged)
delta_E_rms = np.sqrt(np.mean(np.array(delta_E_list)**2))
print(f"\n--- Goal 3 Results (N={N}, ε={epsilon}) ---")
print(f"RMS energy change: ⟨(ΔE)²⟩^(1/2) = {delta_E_rms:.4f}")
print(f"Mean |ΔE| = {np.mean(delta_E_list):.4f} ± {np.std(delta_E_list):.4f}")
print(f"Standard protocol: {M} disorder realizations, each with independent perturbation")

Saved perturbed couplings to generated_data/Jij_N50_seed0_pert10000_eps0.01.csv
N = 50
Energy = -33.99566427486773
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed0_pert10000_eps0.01.csv
Seed 0, Pert 10000: E_orig = -33.9528, E_pert = -33.9957, |ΔE| = 0.0429
Saved perturbed couplings to generated_data/Jij_N50_seed1_pert10001_eps0.01.csv
N = 50
Energy = -33.99566427486773
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed0_pert10000_eps0.01.csv
Seed 0, Pert 10000: E_orig = -33.9528, E_pert = -33.9957, |ΔE| = 0.0429
Saved perturbed couplings to generated_data/Jij_N50_seed1_pert10001_eps0.01.csv
N = 50
Energy = -36.216823236041954
Status: integer optimal, tolerance
MIP gap: 0.0001
Saved solution to cplex_solutions/solution_N50_seed1_pert10001_eps0.01.csv
Seed 1, Pert 10001: E_orig = -36.2220, E_pert = -36.2168, |ΔE| = 0.0051
Saved perturbed couplings to generated_data/Jij_N50_seed2_pe

In [12]:
# Save Goal 3 results for chaos exponent analysis
results_df = pd.DataFrame({
    'seed_orig': range(M),
    'seed_pert': [10000 + i for i in range(M)],
    'E_orig': energy_orig_list,
    'E_pert': energy_pert_list,
    'delta_E': delta_E_list
})

os.makedirs('chaos_results', exist_ok=True)
results_df.to_csv(f'chaos_results/chaos_N{N}_eps{epsilon}.csv', index=False)
print(f"\nSaved chaos results to chaos_results/chaos_N{N}_eps{epsilon}.csv")
print(f"Protocol: {M} disorder realizations × 1 perturbation each = {M} samples")


Saved chaos results to chaos_results/chaos_N50_eps0.01.csv
Protocol: 40 disorder realizations × 1 perturbation each = 40 samples


In [ ]:
# N= [50, 500, 100]
# rms_energy_change = [0.0467, 0.1652, 0.2120]

## Test Different Scaling Laws for ⟨(ΔE)²⟩^(1/2)